In [1]:
import numpy as np
from helpers import *
import matplotlib.pyplot as plt
import cv2
import scipy.optimize

In [2]:
t1 = np.array([[0, 0, 1]]).T
K1,T1,P1 = make_camera(700,600,400,np.eye(3),t1)
t2 = np.array([[0, 0, 20]]).T
K2,T2,P2 = make_camera(700,600,400,np.eye(3),t2)

In [3]:
Q = np.array([[1, 1, 0, 1]]).T
q1 = shom((P1@Q).T)
q2 = shom((P2@Q).T)
q1,q2

(array([[1.3e+03, 1.1e+03, 1.0e+00]]), array([[635., 435.,   1.]]))

In [4]:
Ps = [P1,P2]
Qs = np.concatenate([q1,q2],axis=0)
((triangulate(Qs,Ps) - Q.T) < 1e-8).all()

True

In [5]:
q1_tilda = q1 + np.array([[1., -1., 0.]])
q2_tilda = q2 + np.array([[1., -1., 0.]])
Qs_tilda = np.concatenate([q1_tilda,q2_tilda],axis=0)
Q_tilda = triangulate(Qs_tilda,Ps)
rep_q1_tilda = shom(P1 @ Q_tilda.T)
rep_q2_tilda = shom(P2 @ Q_tilda.T)
# this is how far the reprojected point is from the originally observed
np.linalg.norm(rep_q1_tilda - q1_tilda), np.linalg.norm(rep_q2_tilda - q2_tilda)

(13.43301898819186, 0.6717725840473774)

In [6]:
# 3D error of noisy point in the images
np.linalg.norm(Q_tilda - Q)

2.449730193515837

In [7]:
Q_hat = shom(triangulate_nonlin(Qs_tilda,Ps))
rep_q1_hat = shom(P1 @ Q_hat.T)
rep_q2_hat = shom(P2 @ Q_hat.T)
# this is how far the reprojected point is from the originally observed
np.linalg.norm(rep_q1_hat - q1_tilda), np.linalg.norm(rep_q2_hat - q2_tilda), np.linalg.norm(Q_hat - Q)

(0.06701027274072646, 1.3401508667025752, 2.4494758670155377)